In [1]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [2]:
# SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
scopes = ['https://www.googleapis.com/auth/calendar.readonly']
def getService():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    

    flow = InstalledAppFlow.from_client_secrets_file("/Users/ayadav/Downloads/credentiallyell.json", scopes=scopes)
    credentials = flow.run_console()

    pickle.dump(credentials, open("token.pkl", "wb")) 
    credentials = pickle.load(open("token.pkl", "rb"))

    service = build("calendar", "v3", credentials=credentials)
    return service

In [3]:
service = getService()
def getEvents(service, calId, startTime, endTime):

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
    #print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId=calId, timeMin=startTime, timeMax=endTime
                                        ,singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(start, event['summary'])
        
    return events

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=552436079146-ggcdfn9hhhevoa6o8se6b8crhgo6e8ap.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=s18CcIuDCa0HNo1Eev1LjMLzZWrNZJ&prompt=consent&access_type=offline
Enter the authorization code: 4/1gH7y6qKVX-yAz4azKKCGmGvGLuoQyPBh7ZxldcJshs-6AAJYAff2yw


In [8]:
import datetime
from pytz import timezone
pst_tz = timezone('US/Pacific')

d1 = datetime.datetime(2020 ,7 ,6,0 ,0 ,0, tzinfo=pst_tz)  #(datetime.timezone.utc).isoformat()
startTime = d1.isoformat()
print(startTime)

2020-07-06T00:00:00-07:53


In [9]:
d2=datetime.datetime(2020 ,7 ,12 ,23 ,59 ,59, tzinfo=pst_tz)  #(datetime.timezone.utc).isoformat()
endTime = d2.isoformat()
print(endTime)

2020-07-12T23:59:59-07:53


In [10]:
# Get calander events for 5th floor lab request
calEvents5thFloor = getEvents(service ,'lyell.com_kid4qglss8sfjfv82j48qcgbn0@group.calendar.google.com',startTime,endTime)
## Get calander events for 2th floor lab request
calEvents2ndFloor = getEvents(service ,'lyell.com_bpsf6a640rmqf760bm3jkga9e4@group.calendar.google.com',startTime,endTime)

2020-07-06T07:00:00-07:00 Ben TCEB
2020-07-06T08:00:00-07:00 Tracy (TCEB) TC
2020-07-06T08:00:00-07:00 Glenna (PCE)
2020-07-06T08:00:00-07:00 Cindy PCE
2020-07-06T08:15:00-07:00 Sharon PCE Mol Bio
2020-07-06T09:00:00-07:00 Shipping & Receiving- Noel 
2020-07-06T09:00:00-07:00 Spencer (TCEB) - TC
2020-07-06T09:00:00-07:00 Tad PCE TC/Bench
2020-07-06T09:00:00-07:00 Aye (PCE) 
2020-07-06T10:00:00-07:00 Esha TC (TCEB)
2020-07-06T10:00:00-07:00 Nicole
2020-07-06T11:30:00-07:00 Sydney - TCEB
2020-07-06T12:00:00-07:00 Leonard PCE TC
2020-07-06T12:00:00-07:00 Howell PCE TC 
2020-07-06T12:30:00-07:00 Leah PCE TC and flow
2020-07-06T13:00:00-07:00 Megan M.- TC (TCEB)
2020-07-06T13:00:00-07:00 Emi (TCEB) TC
2020-07-07T07:00:00-07:00 Ben TCEB
2020-07-07T08:00:00-07:00 Tracy (TCEB) TC
2020-07-07T08:00:00-07:00 Cindy PCE
2020-07-07T09:00:00-07:00 Shipping & Receiving- Noel 
2020-07-07T09:00:00-07:00 Tad PCE TC/Bench
2020-07-07T09:00:00-07:00 Sydney (TCEB)
2020-07-07T09:00:00-07:00 Courtney (TCEB) - 

In [11]:
#important code
import dateutil
from dateutil import parser
def processCalInvites(eventList):
    newRows = []
    for row in eventList:
        date = dateutil.parser.parse(row['start']['dateTime'])
        startHour = date.hour
        stopHour = dateutil.parser.parse(row['end']['dateTime']).hour
        day = date.day
        row['meeting_time'] = str(date.month) + '/' + str(day) + '/' + str(date.year)  + ' '+ str(startHour)
        
        if startHour < stopHour:
            while startHour < stopHour:
                startHour = startHour + 1
                newRow = row.copy()
                newRow['meeting_time'] = str(date.month) + '/' + str(day) + '/' + str(date.year)  + ' '+ str(startHour)
                newRows.append(newRow)
        else:
            while stopHour < startHour:
                startHour = startHour - 1
                newRow = row.copy()
                newRow['meeting_time'] = str(date.month) + '/' + str(day) + '/' + str(date.year)  + ' '+ str(startHour)
                newRows.append(newRow)
                
    eventList.extend(newRows)

In [12]:
processCalInvites(calEvents2ndFloor)
processCalInvites(calEvents5thFloor)


import pandas as pd
df_calEvents5thFloor= pd.DataFrame(calEvents5thFloor)
df_calEvents2ndFloor = pd.DataFrame(calEvents2ndFloor)
df = pd.DataFrame()
df = pd.concat([df_calEvents5thFloor,df_calEvents2ndFloor])

In [13]:
## Call processCalInvites to convert meeting time into houly blocks
print(len(df))
print(len(df_calEvents5thFloor))
print(len(df_calEvents2ndFloor))


739
421
318


In [14]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
df.to_csv("/Users/ayadav/Desktop/" + startTime + "-" + endTime + "raw.csv")

In [15]:
#df['start'][0]
df.dtypes

kind                 object
etag                 object
id                   object
status               object
htmlLink             object
created              object
updated              object
summary              object
creator              object
organizer            object
start                object
end                  object
iCalUID              object
sequence              int64
reminders            object
meeting_time         object
attendees            object
description          object
colorId              object
recurringEventId     object
originalStartTime    object
guestsCanModify      object
dtype: object

In [34]:
df['start'] = df.apply(lambda row: pd.to_datetime(row.start['dateTime']), axis = 1)
df['end'] = df.apply(lambda row: pd.to_datetime(row.end['dateTime']), axis = 1)
creator = list(df['creator'])    # extract the list of dictionaries
df['creator'] = pd.DataFrame(creator).astype(str) 

AttributeError: 'Series' object has no attribute 'start'

In [26]:
df

creator      
                                                     <lambda_0> count
meeting_time                                                         
7/10/2020 0                                   hjensen@lyell.com     1
7/10/2020 1                                   hjensen@lyell.com     1
7/10/2020 10  tooi@lyell.com,bpickett@lyell.com,gfoight@lyel...    14
7/10/2020 11  hjensen@lyell.com,bpickett@lyell.com,gfoight@l...    15
7/10/2020 12  eafreen@lyell.com,kchickering@lyell.com,lngo@l...    12
7/10/2020 13  lchen@lyell.com,eafreen@lyell.com,kchickering@...    12
7/10/2020 14  hjensen@lyell.com,eafreen@lyell.com,hjensen@ly...    10
7/10/2020 15  eafreen@lyell.com,hjensen@lyell.com,lchen@lyel...     8
7/10/2020 16  hjensen@lyell.com,hjensen@lyell.com,hjensen@ly...     4
7/10/2020 17  hjensen@lyell.com,hjensen@lyell.com,bpickett@l...     3
7/10/2020 18               hjensen@lyell.com,bpickett@lyell.com     2
7/10/2020 19                               sspadinger@lyell.com     1
7/10/2020 2                                   hjensen@lyell.com     1
7/10/2020 3                                   hjensen@lyell.com     1
7/10/2020 4                                   hjensen@lyell.com     1
7/10/2020 5                                   hjensen@lyell.com     1
7/10/2020 6                                   hjensen@lyell.com     1
7/10/2020 7   bpickett@lyell.com,bpickett@lyell.com,hjensen@...     3
7/10/2020 8   gfoight@lyell.com,cyang@lyell.com,bpickett@lye...     6
7/10/2020 9   eafreen@lyell.com,kchickering@lyell.com,lngo@l...    13
7/11/2020 10                                csimianer@lyell.com     1
7/11/2020 11                                  hjensen@lyell.com     1
7/11/2020 12                                  hjensen@lyell.com     1
7/11/2020 13                                  hjensen@lyell.com     1
7/11/2020 14                                  hjensen@lyell.com     1
7/11/2020 15                              kchickering@lyell.com     1
7/12/2020 19                                  hjensen@lyell.com     1
7/12/2020 20                              kchickering@lyell.com     1
7/12/2020 21                              kchickering@lyell.com     1
7/12/2020 22                              kchickering@lyell.com     1
7/12/2020 23                                     lngo@lyell.com     1
7/6/2020 0                                kchickering@lyell.com     1
7/6/2020 1                                        ske@lyell.com     1
7/6/2020 10   eafreen@lyell.com,nzhang@lyell.com,bpickett@ly...    16
7/6/2020 11   sspadinger@lyell.com,bpickett@lyell.com,tooi@l...    16
7/6/2020 12   lchen@lyell.com,hmoffett@lyell.com,ltait@lyell...    18
7/6/2020 13   mmurt@lyell.com,etokuda@lyell.com,ske@lyell.co...    19
7/6/2020 14   ske@lyell.com,spark@lyell.com,tdavenport@lyell...    16
7/6/2020 15   ske@lyell.com,spark@lyell.com,tdavenport@lyell...    14
7/6/2020 16   ske@lyell.com,tdavenport@lyell.com,ske@lyell.c...    11
7/6/2020 17   ske@lyell.com,tdavenport@lyell.com,ske@lyell.c...     9
7/6/2020 18                   bpickett@lyell.com,tooi@lyell.com     2
7/6/2020 19                  spark@lyell.com,bpickett@lyell.com     2
7/6/2020 2                                        ske@lyell.com     1
7/6/2020 20                                  bpickett@lyell.com     1
7/6/2020 3                                        ske@lyell.com     1
7/6/2020 4                                        ske@lyell.com     1
7/6/2020 5                                        ske@lyell.com     1
7/6/2020 6                                        ske@lyell.com     1
7/6/2020 7    bpickett@lyell.com,bpickett@lyell.com,tooi@lye...     4
7/6/2020 8    tooi@lyell.com,gfoight@lyell.com,cyang@lyell.c...     9
7/6/2020 9    kchickering@lyell.com,spark@lyell.com,tdavenpo...    14
7/7/2020 0                                      ltait@lyell.com     1
7/7/2020 1                                      ltait@lyell.com     1
7/7/2020 10   eafreen@lyell.com,bpickett@lyell.com,tooi@lyel...   

In [27]:
# df['meeting_time']= df.apply(lambda row:  str(row.start.month) + '-' + str(row.start.day) + '-' + str(row.start.hour) , axis = 1)  

In [28]:
hourAgg = df.groupby('meeting_time', as_index=False).agg({'creator': [lambda x: ','.join(x), "count"]})

KeyError: 'creator'

In [30]:
df_plot.drop(columns='count').plot(kind='scatter', stacked=True)

NameError: name 'df_plot' is not defined

In [31]:
hourAgg

meeting_time                                            creator      
                                                          <lambda_0> count
0     7/10/2020 0                                  hjensen@lyell.com     1
1     7/10/2020 1                                  hjensen@lyell.com     1
2    7/10/2020 10  tooi@lyell.com,bpickett@lyell.com,gfoight@lyel...    14
3    7/10/2020 11  hjensen@lyell.com,bpickett@lyell.com,gfoight@l...    15
4    7/10/2020 12  eafreen@lyell.com,kchickering@lyell.com,lngo@l...    12
5    7/10/2020 13  lchen@lyell.com,eafreen@lyell.com,kchickering@...    12
6    7/10/2020 14  hjensen@lyell.com,eafreen@lyell.com,hjensen@ly...    10
7    7/10/2020 15  eafreen@lyell.com,hjensen@lyell.com,lchen@lyel...     8
8    7/10/2020 16  hjensen@lyell.com,hjensen@lyell.com,hjensen@ly...     4
9    7/10/2020 17  hjensen@lyell.com,hjensen@lyell.com,bpickett@l...     3
10   7/10/2020 18               hjensen@lyell.com,bpickett@lyell.com     2
11   7/10/2020 19                               sspadinger@lyell.com     1
12    7/10/2020 2                                  hjensen@lyell.com     1
13    7/10/2020 3                                  hjensen@lyell.com     1
14    7/10/2020 4                                  hjensen@lyell.com     1
15    7/10/2020 5                                  hjensen@lyell.com     1
16    7/10/2020 6                                  hjensen@lyell.com     1
17    7/10/2020 7  bpickett@lyell.com,bpickett@lyell.com,hjensen@...     3
18    7/10/2020 8  gfoight@lyell.com,cyang@lyell.com,bpickett@lye...     6
19    7/10/2020 9  eafreen@lyell.com,kchickering@lyell.com,lngo@l...    13
20   7/11/2020 10                                csimianer@lyell.com     1
21   7/11/2020 11                                  hjensen@lyell.com     1
22   7/11/2020 12                                  hjensen@lyell.com     1
23   7/11/2020 13                                  hjensen@lyell.com     1
24   7/11/2020 14                                  hjensen@lyell.com     1
25   7/11/2020 15                              kchickering@lyell.com     1
26   7/12/2020 19                                  hjensen@lyell.com     1
27   7/12/2020 20                              kchickering@lyell.com     1
28   7/12/2020 21                              kchickering@lyell.com     1
29   7/12/2020 22                              kchickering@lyell.com     1
30   7/12/2020 23                                     lngo@lyell.com     1
31     7/6/2020 0                              kchickering@lyell.com     1
32     7/6/2020 1                                      ske@lyell.com     1
33    7/6/2020 10  eafreen@lyell.com,nzhang@lyell.com,bpickett@ly...    16
34    7/6/2020 11  sspadinger@lyell.com,bpickett@lyell.com,tooi@l...    16
35    7/6/2020 12  lchen@lyell.com,hmoffett@lyell.com,ltait@lyell...    18
36    7/6/2020 13  mmurt@lyell.com,etokuda@lyell.com,ske@lyell.co...    19
37    7/6/2020 14  ske@lyell.com,spark@lyell.com,tdavenport@lyell...    16
38    7/6/2020 15  ske@lyell.com,spark@lyell.com,tdavenport@lyell...    14
39    7/6/2020 16  ske@lyell.com,tdavenport@lyell.com,ske@lyell.c...    11
40    7/6/2020 17  ske@lyell.com,tdavenport@lyell.com,ske@lyell.c...     9
41    7/6/2020 18                  bpickett@lyell.com,tooi@lyell.com     2
42    7/6/2020 19                 spark@lyell.com,bpickett@lyell.com     2
43     7/6/2020 2                                      ske@lyell.com     1
44    7/6/2020 20                                 bpickett@lyell.com     1
45     7/6/2020 3                                      ske@lyell.com     1
46     7/6/2020 4                                      ske@lyell.com     1
47     7/6/2020 5                                      ske@lyell.com     1
48     7/6/2020 6                                      ske@lyell.com     1
49     7/6/2020 7  bpickett@lyell.com,bpickett@lyell.com,tooi@lye...     4
50     7/6/2020 8  tooi@lyell.com,gfoight@lyell.com,cyang@lyell.c...     9
51     7/6/2020 9  kchickering

In [32]:
# save csv file
df_csv = hourAgg.to_csv(startTime + '-' + endTime + 'agg-lab-bookings.csv')